### FSEOF analysis


In [1]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite
from cameo.strain_design.deterministic.flux_variability_based import FSEOF

In [3]:
model = read_sbml_model('data/yeastGEM_het.xml')

In [ ]:
fseof = FSEOF(model)

In [ ]:
fseofresult=fseof.run(target=model.reactions.R07215)

In [ ]:
fseofresult

In [ ]:
from matplotlib import pyplot as plt
fseofresult.data_frame[140:].T.plot()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

From the FSEOF analysis, we can see that reaction r_4460 is negative. This suggests that a decrease in this reaction is correlated with an increase in production of 7-dehydrocholersterol. This reaction is the phosphate transport in via proton symport, which suggests that phosphate limitation might be productive for the optimization of our cell factory. 

We can therefore try to knock out this reaction and see if it changes anything, but we can also try to remove some of the phosphate from the environment.

In [ ]:
from cobra.flux_analysis.phenotype_phase_plane import production_envelope, add_envelope
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 

In [ ]:
# Phosphate transport knockout
with model:    
    print("Normal growth conditions")
    print("Maximal growth rate:")
    print(model.slim_optimize())
    model.objective = model.reactions.R07215
    print("Maximal 7-dehydrocholrsterol production rate")
    print(model.slim_optimize())

with model:
    model.reactions.r_4460.knock_out()
    
    
    print("\nWithout phosphate")
    print("Maximal growth rate:")
    print(model.slim_optimize())
    model.objective = model.reactions.R07215
    print("Maximal 7-dehydrocholrsterol production rate")
    print(model.slim_optimize())
    
    ppp_7DH = production_envelope(model,
                    reactions=[model.reactions.R07215],
                    objective=model.reactions.r_2111)
    pppp = ppp_7DH.plot(x='R07215', y='flux_maximum')
    fig = pppp.get_figure()
    plt.title(label="Phosphate knockout")

In [ ]:
# Phosphate limitation in the media

with model:    
    print("Normal growth conditions")
    print("Maximal growth rate:")
    print(model.slim_optimize())
    model.objective = model.reactions.R07215
    print("Maximal 7-dehydrocholrsterol production rate")
    print(model.slim_optimize())

with model:
    medium = model.medium
    medium['r_2005'] = 0.1    
    model.medium = medium
    
    
    print("\nWithout phosphate")
    print("Maximal growth rate:")
    print(model.slim_optimize())
    model.objective = model.reactions.R07215
    print("Maximal 7-dehydrocholrsterol production rate")
    print(model.slim_optimize())
    
    ppp_7DH = production_envelope(model,
                    reactions=[model.reactions.R07215],
                    objective=model.reactions.r_2111)
    pppp = ppp_7DH.plot(x='R07215', y='flux_maximum')
    fig = pppp.get_figure()
    plt.title(label="Phosphate minimization")

From this PPP, we can see our first deviation from the straight line that signals the trade-off between growth and production. We have now a better chance of finding an optimization route

In [4]:
from cameo.strain_design import OptGene

with model:
    medium = model.medium
    medium['r_2005'] =1    
    model.medium = medium
    with model:
        optgene = OptGene(model)
        result_phoslim1 = optgene.run(target=model.reactions.R07215,biomass=model.reactions.r_2111,substrate=model.reactions.r_1714,max_evaluations=2000,plot=False)

Starting optimization at Sun, 29 Nov 2020 21:18:22


HBox()

Finished after 01:09:23


In [5]:
result_phoslim1

,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness
